<a href="https://colab.research.google.com/github/pszachew/Music-Genre-Classification/blob/master/training_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
X = pd.read_csv('/content/gdrive/MyDrive/csv_data/embedded.csv',header=None)

In [4]:
y = pd.read_csv('/content/gdrive/MyDrive/csv_data/labels.csv', header=None)

In [5]:
scaler = StandardScaler()
scaler.fit(X)
X_sc =pd.DataFrame(scaler.transform(X))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_sc,y,test_size=0.3, random_state=10)

## **Logistic Regression**

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
log_reg_mod = LogisticRegression(max_iter=1000)

In [9]:
log_reg_mod.fit(X_train, np.array(y_train).flatten())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
joblib.dump(log_reg_mod, 'log_reg.sav')

['log_reg.sav']

In [11]:
y_pre_lg = log_reg_mod.predict(X_test)

In [12]:
print(classification_report(y_test,y_pre_lg))

              precision    recall  f1-score   support

       blues       0.87      0.94      0.90        35
   classical       0.89      1.00      0.94        33
     country       0.81      0.78      0.79        27
       disco       0.88      0.68      0.77        34
      hiphop       0.77      0.96      0.85        24
        jazz       1.00      0.91      0.96        35
       metal       0.97      1.00      0.98        31
         pop       0.77      0.88      0.82        26
      reggae       0.92      0.80      0.86        30
        rock       0.70      0.64      0.67        25

    accuracy                           0.86       300
   macro avg       0.86      0.86      0.85       300
weighted avg       0.87      0.86      0.86       300



In [13]:
print(confusion_matrix(y_test, y_pre_lg))

[[33  1  0  0  0  0  0  0  0  1]
 [ 0 33  0  0  0  0  0  0  0  0]
 [ 2  0 21  2  0  0  0  1  0  1]
 [ 0  1  1 23  2  0  0  4  0  3]
 [ 0  0  0  0 23  0  0  0  1  0]
 [ 1  2  0  0  0 32  0  0  0  0]
 [ 0  0  0  0  0  0 31  0  0  0]
 [ 0  0  0  0  2  0  0 23  0  1]
 [ 0  0  1  1  1  0  0  2 24  1]
 [ 2  0  3  0  2  0  1  0  1 16]]


In [14]:
kf3 = KFold(n_splits=3, shuffle=True)
mod_lg = LogisticRegression(max_iter=10000)

In [15]:
i=1
for x_train_idx, x_test_idx in kf3.split(X_sc):
  X_train = X_sc.iloc[x_train_idx]
  X_test = X_sc.iloc[x_test_idx]
  y_train = y.iloc[x_train_idx]
  y_test = y.iloc[x_test_idx]
  mod_lg.fit(X_train, np.array(y_train).flatten())
  print(f"Accuracy for the fold no. {i} on the test set: {accuracy_score(y_test, mod_lg.predict(X_test))}")
  i=i+1




Accuracy for the fold no. 1 on the test set: 0.8618618618618619
Accuracy for the fold no. 2 on the test set: 0.8678678678678678
Accuracy for the fold no. 3 on the test set: 0.8318318318318318


## **KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
neigh = KNeighborsClassifier(n_neighbors = 4)
neigh.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [ ]:
acc_ls = []

In [ ]:
for n in range (3,12):
  neigh = KNeighborsClassifier(n_neighbors = n)
  neigh.fit(X_train, y_train)
  acc_ls.append(accuracy_score(y_test, neigh.predict(X_test)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A colum

In [ ]:
acc_ls

[0.8033333333333333,
 0.81,
 0.79,
 0.8,
 0.7933333333333333,
 0.8033333333333333,
 0.81,
 0.79,
 0.8,
 0.7933333333333333,
 0.8,
 0.7833333333333333,
 0.78,
 0.7933333333333333]